In [0]:
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://files@stazure2025.dfs.core.windows.net/bronze/Orders/")
df.display()

In [0]:
from pyspark.sql.functions import *

df=df.withColumn("PickedByPersonID",col("PickedByPersonID").cast("integer"))
df=df.withColumn("BackorderOrderID",col("BackorderOrderID").cast("integer"))
df=df.withColumn("PickingCompletedWhen",to_timestamp(col("CustomerPurchaseOrderNumber")))


In [0]:
df.write \
  .mode("overwrite") \
  .format("delta") \
  .option("mergeSchema", "true")\
  .option("delta.enableChangeDataFeed", "true") \
  .saveAsTable("project.bronze.Orders")

In [0]:
df.count()

In [0]:
df_sliver=spark.sql(f"select * from project.bronze.Orders")
df_sliver.display()

In [0]:
df_sliver.columns

In [0]:
df_sliver=df_sliver.drop('PickedByPersonID','BackorderOrderID','Comments','DeliveryInstructions','InternalComments','PickingCompletedWhen')

In [0]:
df_sliver.columns

In [0]:
df_sliver.count

In [0]:
df_sliver.write.format("delta").mode("append").option("mergeSchema", "true").option("delta.enableChangeDataFeed", "true").saveAsTable("project.silver.oooo")

In [0]:
df_gold=df_sliver.write.format("delta").mode("append").option("mergeSchema", "true").option("delta.enableChangeDataFeed", "true").saveAsTable("project.gold.live")

In [0]:
%sql
MERGE INTO project.gold.finaly g
USING project.gold.live s
ON g.OrderID = s.OrderID
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;
